In [1]:
!pip install isodate

In [2]:
from pyspark.sql.types import *
import isodate

# File location
file_location = "/FileStore/tables/recipes.json"
df = spark.read.json(file_location)

display(df.limit(20))

cookTime,datePublished,description,image,ingredients,name,prepTime,recipeYield,url
PT,2013-04-01,Got leftover Easter eggs? Got leftover Easter ham? Got a hearty appetite? Good! You've come to the right place! I...,http://static.thepioneerwoman.com/cooking/files/2013/03/leftoversandwich.jpg,"12 whole Hard Boiled Eggs 1/2 cup Mayonnaise 3 Tablespoons Grainy Dijon Mustard Salt And Pepper, to taste Several Dashes Worcestershire Sauce Leftover Baked Ham, Sliced Kaiser Rolls Or Other Bread Extra Mayonnaise And Dijon, For Spreading Swiss Cheese Or Other Cheese Slices Thinly Sliced Red Onion Avocado Slices Sliced Tomatoes Lettuce, Spinach, Or Arugula",Easter Leftover Sandwich,PT15M,8,http://thepioneerwoman.com/cooking/2013/04/easter-leftover-sandwich/
PT10M,2011-06-06,I finally have basil in my garden. Basil I can use. This is a huge development. I had no basil during the winter. None. G...,http://static.thepioneerwoman.com/cooking/files/2011/06/pesto.jpg,"3/4 cups Fresh Basil Leaves 1/2 cup Grated Parmesan Cheese 3 Tablespoons Pine Nuts 2 cloves Garlic, Peeled Salt And Pepper, to taste 1/3 cup Extra Virgin Olive Oil 1/2 cup Heavy Cream 2 Tablespoons Butter 1/4 cup Grated Parmesan (additional) 12 ounces, weight Pasta (cavitappi, Fusili, Etc.) 2 whole Tomatoes, Diced",Pasta with Pesto Cream Sauce,PT6M,8,http://thepioneerwoman.com/cooking/2011/06/pasta-with-pesto-cream-sauce/
PT15M,2011-09-15,"This was yummy. And easy. And pretty! And it took basically no time to make. Before I share the recipe, I'll just say it:...",http://static.thepioneerwoman.com/cooking/files/2011/09/porkloin.jpg,"2 whole Pork Tenderloins Salt And Pepper, to taste 8 Tablespoons Herbs De Provence (more If Needed 1 cup Preserves (fig, Peach, Plum) 1 cup Water 1 Tablespoon Vinegar",Herb Roasted Pork Tenderloin with Preserves,PT5M,12,http://thepioneerwoman.com/cooking/2011/09/herb-roasted-pork-tenderloin-with-preserves/
PT20M,2012-04-23,"I made this for a late lunch Saturday, and it absolutely completed me. It also absolutely completed my fourteen-year-old daug...",http://static.thepioneerwoman.com/cooking/files/2012/04/florentine.jpg,"1 pound Penne 4 whole Boneless, Skinless Chicken Breasts Salt And Pepper, to taste 2 Tablespoons Butter 2 Tablespoons Olive Oil 4 cloves Garlic, Minced 3/4 cups Dry White Wine 3/4 cups Low-sodium Broth, More If Needed 1 bag Baby Spinach 2 cups Grape Tomatoes, Halved Lengthwise 4 ounces, weight Parmesan Cheese, Shaved With Vegetable Peeler",Chicken Florentine Pasta,PT10M,10,http://thepioneerwoman.com/cooking/2012/04/chicken-florentine-pasta/
PT,2011-06-13,"Iced coffee is my life. When I wake up, often around the time party animals on the west coast are just heading home, I start ...",http://static.thepioneerwoman.com/cooking/files/2011/06/icedcoffee.jpg,"1 pound Ground Coffee (good, Rich Roast) 8 quarts Cold Water Half-and-half (healthy Splash Per Serving) Sweetened Condensed Milk (2-3 Tablespoons Per Serving) Note: Can Use Skim Milk, 2% Milk, Whole Milk, Sugar, Artificial Sweeteners, Syrups...adapt To Your Liking!",Perfect Iced Coffee,PT8H,24,http://thepioneerwoman.com/cooking/2011/06/perfect-iced-coffee/
PT10M,2012-05-31,"When I was in Albuquerque with Marlboro Man and the boys a month ago, I had a really fun book signing. Such incredibly nice a...",http://static.thepioneerwoman.com/cooking/files/2012/05/enchilada.jpg,"1 whole Onion, Diced 2 Tablespoons Butter 1 can (15 Ounce) Green Enchilada Sauce 2 cans (4 Ounce) Chopped Green Chilies 12 whole Corn Tortillas 2 cups Freshly Grated Cheddar (or Cheddar-jack) Cheese (or Any Cheese You'd Like) Sour Cream Salsa Pico De Gallo (optional) Guacamole (optional) Cilantro Leaves, Optional",Easy Green Chile Enchiladas,PT5M,4,http://thepioneerwoman.com/cooking/2012/05/easy-green-chile-enchiladas/
PT5M,2013-03-25,"Imagine the Easter Bunny laying an egg. Wait. That’s not anatomically possible. And anyway, the Easter Bunny is a b...",http://static.thepioneerwoman.com/cooking/files/2013/03/DSC_9637.jpg,

In [3]:
def duration_converter(time):
  if time != '':
    return isodate.parse_duration(time).seconds / 60
  else:
    return 0.0

cook_time = [duration_converter(row.cookTime) for row in df.select("cookTime").rdd.collect()]
prep_time = [duration_converter(row.prepTime) for row in df.select("prepTime").rdd.collect()]

In [4]:
schema=StructType([StructField("cook_time", FloatType()),
                   StructField("prep_time", FloatType())])
result=spark.createDataFrame(zip(cook_time, prep_time), schema=schema) 
result.show()

+---------+---------+
cook_time|prep_time|
+---------+---------+
 0.0| 15.0|
 10.0| 6.0|
 15.0| 5.0|
 20.0| 10.0|
 0.0| 480.0|
 10.0| 5.0|
 5.0| 20.0|
 25.0| 10.0|
 2.0| 25.0|
 15.0| 5.0|
 75.0| 10.0|
 60.0| 15.0|
 10.0| 15.0|
 20.0| 30.0|
 10.0| 20.0|
 5.0| 20.0|
 0.0| 10.0|
 45.0| 10.0|
 60.0| 360.0|
 15.0| 10.0|
+---------+---------+
only showing top 20 rows

In [5]:
from pyspark.sql.functions import *
result = result.withColumn('total_time', expr("cook_time + prep_time"))

In [6]:
result.show()

+---------+---------+----------+
cook_time|prep_time|total_time|
+---------+---------+----------+
 0.0| 15.0| 15.0|
 10.0| 6.0| 16.0|
 15.0| 5.0| 20.0|
 20.0| 10.0| 30.0|
 0.0| 480.0| 480.0|
 10.0| 5.0| 15.0|
 5.0| 20.0| 25.0|
 25.0| 10.0| 35.0|
 2.0| 25.0| 27.0|
 15.0| 5.0| 20.0|
 75.0| 10.0| 85.0|
 60.0| 15.0| 75.0|
 10.0| 15.0| 25.0|
 20.0| 30.0| 50.0|
 10.0| 20.0| 30.0|
 5.0| 20.0| 25.0|
 0.0| 10.0| 10.0|
 45.0| 10.0| 55.0|
 60.0| 360.0| 420.0|
 15.0| 10.0| 25.0|
+---------+---------+----------+
only showing top 20 rows

In [7]:
#removing the missing values which has total time 0
result = result.filter(result.total_time != 0)

In [8]:
#easy
result.where(result.total_time < 30).select(avg('total_time')).show()

+-----------------+
 avg(total_time)|
+-----------------+
17.11520737327189|
+-----------------+

In [9]:
#medium
result.where((result.total_time >= 30) & (result.total_time < 60)).select(avg('total_time')).show()

+-----------------+
 avg(total_time)|
+-----------------+
39.37065637065637|
+-----------------+

In [10]:
#hard
result.where(result.total_time >= 60).select(avg('total_time')).show()

+------------------+
 avg(total_time)|
+------------------+
134.51340996168582|
+------------------+